# Noisy Channel Model Implementation for Machine Translation

#### Based on Research Paper

### Loading the test dataset

In [4]:
# Load a few English sentences (source for translation)
test_file = r"D:\NLP Nosiy Machine Translation\datasets\training-parallel-europarl-v7\training\europarl-v7.de-en.en"  # adjust the path if necessary
with open(test_file, encoding="utf-8") as f:
    test_sentences = [line.strip() for line in f.readlines()[:10]]  # first 10 sentences

print("Loaded test sentences:")
for s in test_sentences:
    print(s)

Loaded test sentences:
Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
You have requested a debate on this subject in the course of the next few days, during this part-session.
In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.
Please rise, then, for this minute' s silence.
(The House rose and observed a minute' s silence)
Madam President, on a point of order.
You will be aware from the press and television that there have been a number

#### Loading the pretrained models

In [ ]:
from fairseq.models.transformer import TransformerModel

direct_model_path = r"D:\NLP Nosiy Machine Translation\transformers_large_models\wmt19(eng to german)\wmt19.en-de.joined-dict.ensemble"
direct_model_bpe_codes = r":\NLP Nosiy Machine Translation\transformers_large_models\wmt19(eng to german)\wmt19.en-de.joined-dict.ensemble\bpecodes"

channel_model_path = r"D:\NLP Nosiy Machine Translation\transformers_large_models\wmt19(german to english)\wmt19.de-en.joined-dict.ensemble"
channel_model_bpe_codes = r"D:\NLP Nosiy Machine Translation\transformers_large_models\wmt19(german to english)\wmt19.de-en.joined-dict.ensemble\bpecodes"

# Load the direct model (English→German)
direct = TransformerModel.from_pretrained(
    direct_model_path,
    checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',  # ensemble checkpoints separated by ":"
    data_name_or_path=direct_model_path,
    bpe='fastbpe',
    bpe_codes=direct_model_bpe_codes
)

# Load the channel model (German→English)
channel = TransformerModel.from_pretrained(
    channel_model_path,
    checkpoint_file='model.pt',
    data_name_or_path=channel_model_path,
    bpe='fastbpe',
    bpe_codes=channel_model_bpe_codes
)

# # Load the direct model (English→German)
# direct = TransformerModel.from_pretrained(
#     'models/direct/wmt19.en-de.joined-dict.ensemble',
#     checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',  # ensemble checkpoints separated by ":"
#     data_name_or_path='models/direct/wmt19.en-de.joined-dict.ensemble',
#     bpe='fastbpe',
#     bpe_codes='models/direct/wmt19.en-de.joined-dict.ensemble/bpecodes'
# )

# # Load the channel model (German→English)
# channel = TransformerModel.from_pretrained(
#     'models/channel/wmt19.de-en.joined-dict.single_model',
#     checkpoint_file='model.pt',
#     data_name_or_path='models/channel/wmt19.de-en.joined-dict.single_model',
#     bpe='fastbpe',
#     bpe_codes='models/channel/wmt19.de-en.joined-dict.single_model/bpecodes'
# )




### run simple implementation of noisy channel model

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

def gpt2_score(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    log_likelihood = -outputs.loss.item()
    return log_likelihood


In [ ]:
import numpy as np
import torch

def direct_translate(sentence, beam_size=5):
    """Generate a translation using the direct model (P(y|x))."""
    return direct.translate(sentence, beam=beam_size)

def noisy_channel_decode(sentence, beam_size=5, lambda_=1.0, mu=1.0):
    """
    Generate translation candidates with the direct model, then re-rank using
    the channel model (P(x|y)) and language model (P(y)).
    Final score: direct_score + lambda_ * channel_score + mu * lm_score.
    """
    # Generate candidate translations from the direct model
    candidates = direct.translate(sentence, beam=beam_size)
    if isinstance(candidates, str):
        candidates = [candidates]
    
    # Compute scores for each candidate
    direct_scores = [direct.score(candidate) for candidate in candidates]
    channel_scores = [channel.score(candidate, sentence) for candidate in candidates]
    lm_scores = [gpt2_score(candidate) for candidate in candidates]
    
    # Calculate final scores for each candidate
    final_scores = [d + lambda_ * ch + mu * l for d, ch, l in zip(direct_scores, channel_scores, lm_scores)]
    best_idx = np.argmax(final_scores)
    return candidates[best_idx]

# Example usage on a single sentence:
example_sentence = test_sentences[0]
print("Source:", example_sentence)
print("Direct Translation:", direct_translate(example_sentence))
print("Noisy Channel Translation:", noisy_channel_decode(example_sentence))


In [ ]:
print("=== Translations for Test Sentences ===")
for src in test_sentences:
    direct_out = direct_translate(src)
    noisy_out = noisy_channel_decode(src)
    print("Source:", src)
    print("Direct:", direct_out)
    print("Noisy Channel:", noisy_out)
    print("----")


In [ ]:
import sacrebleu

# Load reference translations (German sentences)
ref_file = "data/europarl-v7.de-en.de"  # adjust the path if necessary
with open(ref_file, encoding="utf-8") as f:
    references = [line.strip() for line in f.readlines()[:10]]

# Generate hypothesis translations using the noisy channel decoder
hypotheses = [noisy_channel_decode(src) for src in test_sentences]

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(hypotheses, [references])
print("BLEU score:", bleu.score)
